# 1. Chapter : 상호작용 맵
folium 패키지를 사용하여 상호작용 지도를 만드는 방법

In [1]:
import pandas as pd
import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

## 1-1. 첫 번째 상호작용 맵
`folium.Map()`을 활용한 지도 만들기

In [40]:
# 배경 맵 만들기
m_1 = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=10)
m_1

## 1-2. 데이터 불러오기
범죄 데이터셋 불러오고 맵에 표현하기

In [3]:
# load data
crimes = pd.read_csv("../../01_data/crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin-1')

# 주소가 존재하지 않는 지역의 데이터를 삭제
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# 2018년에 발생한 주요 범죄를 확인
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin(['Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 'Manslaughter'])]

crimes = crimes[crimes.YEAR >= 2018]

# 5개 행 출력
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


## 1-3. 포인트 표현하기
대량의 데이터를 줄이기 위해 9-18시 사이에 발생한 절도범죄 데이터만 사용

In [4]:
daytime_robberies = crimes[(crimes.OFFENSE_CODE_GROUP == 'Robbery') & (crimes.HOUR.isin(range(9, 18)))]
daytime_robberies.head(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"
527,I182070342,381,Robbery,ROBBERY - CAR JACKING,E18,490,NaN,2018-09-01 17:05:00,2018,9,Saturday,17,Part One,CUMMINS HWY,42.276453,-71.112980,"(42.27645319, -71.11297971)"


### 1) folium.Marker 으로 표현하기

In [41]:
# 배경 맵 만들기
m_2 = folium.Map(location=[42.32, -71.0859], tiles='cartodbpositron', zoom_start=13)

# 맵에 포인트 추가하기
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)
    
m_2

### 2) folium.plugins.MarkerCluster 으로 표현하기

In [42]:
# 배경 맵 만들기
m_3 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=13)

# 포인트 추가하기
mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))

m_3.add_child(mc)

m_3

### 3) Bubble 맵으로 표현하기
folium.Circle() 을 통해서 원형의 버블을 생성

In [7]:
daytime_robberies.head(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"
527,I182070342,381,Robbery,ROBBERY - CAR JACKING,E18,490,NaN,2018-09-01 17:05:00,2018,9,Saturday,17,Part One,CUMMINS HWY,42.276453,-71.112980,"(42.27645319, -71.11297971)"


In [8]:
daytime_robberies.iloc[1]

INCIDENT_NUMBER                         I182070342
OFFENSE_CODE                                   381
OFFENSE_CODE_GROUP                         Robbery
OFFENSE_DESCRIPTION          ROBBERY - CAR JACKING
DISTRICT                                       E18
REPORTING_AREA                                 490
SHOOTING                                       NaN
OCCURRED_ON_DATE               2018-09-01 17:05:00
YEAR                                          2018
MONTH                                            9
DAY_OF_WEEK                               Saturday
HOUR                                            17
UCR_PART                                  Part One
STREET                                 CUMMINS HWY
Lat                                      42.276453
Long                                     -71.11298
Location               (42.27645319, -71.11297971)
Name: 527, dtype: object

In [43]:
# 배경맵
m_4 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'
    
# 기본 배경맵에 버블맵 추가하기
for i in range(0, len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)
    
m_4

### 4) Heatmap 만들기
히트맵 지도를 만들기 위해서는 `folium.plugins.HeatMap()`을 사용해야 한다. 이를 통해서 도시에서 발생한 서로 다른 지역간의 범죄의 밀집도를 확인할 수 있다.

In [44]:
# 배경맵 만들기
m_5 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=12)

# 배경맵에 히트맵 추가하기
HeatMap(data=crimes[['Lat', 'Long']], radius=20).add_to(m_5)

m_5

위에 코드를 통해서 `folium.plugins.HeatMap()`을 사용하기 위해서는 아래와 같은 조건이 필요
- 시각 표현을 위해 지리적 위치가 포함된 데이터가 존재해야 함
- `radius`는 히트맵의 표현 강도를 조절 가능함

### 5) Choropleth Maps(등치지역도) 표현하기
지역적으로 발생하는 범죄를 확인하기 위해 등치지역도를 제작

In [11]:
# 보스턴 경찰 경계 데이터 불러오기
districts_full = gpd.read_file("../../01_data/Police_Districts/Police_Districts/Police_Districts.shp")

# 구역으로 인덱스를 재설정
districts = districts_full[['DISTRICT', 'geometry']].set_index('DISTRICT')
districts.head()

,geometry
DISTRICT,
A15,"MULTIPOLYGON (((-71.07416 42.39051, -71.07415 ..."
A7,"MULTIPOLYGON (((-70.99644 42.39557, -70.99644 ..."
A1,"POLYGON ((-71.052 42.36884, -71.05169 42.3687,..."
C6,"POLYGON ((-71.04406 42.35403, -71.04412 42.353..."
D4,"POLYGON ((-71.07416 42.35724, -71.07359 42.357..."


In [12]:
# 구역별로 발생한 범죄 횟수를 시리즈 형태로 변환해서 저장
plot_dict = crimes.DISTRICT.value_counts()

plot_dict.head()

DISTRICT
D4     2885
B2     2231
A1     2130
C11    1899
B3     1421
Name: count, dtype: int64

In [45]:
# 배경맵 생성
m_6 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=12)

# 등치지역도를 배경맵에 추가
Choropleth(geo_data=districts.__geo_interface__,
          data=plot_dict,
          key_on='feature.id',
          fill_color='YlGnBu',
          legend_name='주요 범죄 발생현황 (Jan-Aug 2018)').add_to(m_6)

m_6

`folium.Choropleth()`는 다음과 같은 조건이 요구
- `geo_data`는 지리적 경계를 표현할 수 있어야 함
- `data`는 판다스 시리즈 타입으로 받아야 함
- `key_on`은 항상 `feature.id`를 설정해야 함